# Isn't multiple dispatch just function overloading?

Answer: No. Static types vs actual dynamic types.

In [2]:
abstract type Pet end

struct Dog <: Pet
    name::String
end
struct Cat <: Pet
    name::String
end

function encounter(a::Pet, b::Pet)
    verb = meets(a,b)
    println("$(a.name) meets $(b.name) and $verb.")
end

meets(a::Dog, b::Dog) = "sniffs"
meets(a::Dog, b::Cat) = "chases"
meets(a::Cat, b::Dog) = "hisses"
meets(a::Cat, b::Cat) = "slinks"

meets (generic function with 4 methods)

In [3]:
fido     = Dog("fido")
rex      = Dog("rex")
whiskers = Cat("whiskers")
spots    = Cat("spots")

encounter(fido, rex);
encounter(fido, whiskers);
encounter(whiskers, rex);
encounter(whiskers, spots);

fido meets rex and sniffs.
fido meets whiskers and chases.
whiskers meets rex and hisses.
whiskers meets spots and slinks.


### Alright, let's try the same in, say, C++

```C++
#include <iostream>
#include <string>

using namespace std;

class Pet {
    public:
        string name;
};

string meets(Pet a, Pet b) { return "FALLBACK"; }

void encounter(Pet a, Pet b) {
    string verb = meets(a,b);
    cout << a.name << " meets " << b.name << " and " << verb << endl;
}

class Dog : public Pet {};
class Cat : public Pet {};

string meets(Dog a, Dog b) { return "sniffs"; }
string meets(Dog a, Cat b) { return "chases"; }
string meets(Cat a, Dog b) { return "hisses"; }
string meets(Cat a, Cat b) { return "slinks"; }

int main() {
    Dog fido;     fido.name = "Fido";
    Dog rex;      rex.name = "Rex";
    Cat whiskers; whiskers.name = "Whiskers";
    Cat spots;    spots.name = "Spots";

    encounter(fido, rex);
    encounter(fido, whiskers);
    encounter(whiskers, rex);
    encounter(whiskers, spots);

    return 0;
}
```

### What do you think this does?##

<details>
  <summary>Click to reveal (if Cxx doesn't work for you)</summary>
<br>
    
```
carsten@l91:~/Desktop/Oulu2020/backup/function_overloading$ clang++ pets.cpp -o pets && ./pets

Fido meets Rex and FALLBACK
Fido meets Whiskers and FALLBACK
Whiskers meets Rex and FALLBACK
Whiskers meets Spots and FALLBACK
```
</details>

In [1]:
using Cxx

In [7]:
cxx"""
#include <iostream>
#include <string>

using namespace std;

class Pet {
    public:
        string name;
};

string meets(Pet a, Pet b) { return "FALLBACK"; }

void encounter(Pet a, Pet b) {
    string verb = meets(a,b);
    cout << a.name << " meets " << b.name << " and " << verb << endl;
}

class Dog : public Pet {};
class Cat : public Pet {};

string meets(Dog a, Dog b) { return "sniffs"; }
string meets(Dog a, Cat b) { return "chases"; }
string meets(Cat a, Dog b) { return "hisses"; }
string meets(Cat a, Cat b) { return "slinks"; }

int main() {
    Dog fido;     fido.name = "Fido";
    Dog rex;      rex.name = "Rex";
    Cat whiskers; whiskers.name = "Whiskers";
    Cat spots;    spots.name = "Spots";

    encounter(fido, rex);
    encounter(fido, whiskers);
    encounter(whiskers, rex);
    encounter(whiskers, spots);

    return 0;
}
"""

true

In [12]:
@cxx main()

Fido meets Rex and FALLBACK
Fido meets Whiskers and FALLBACK
Whiskers meets Rex and FALLBACK
Whiskers meets Spots and FALLBACK


0

**What?! Why?**

Templates get you a bit further, but the type information is still only static.